# Get the bpparam for `bpmapply`

## Introduction

In this section, we will show how to use `get_bpparam()` function to get a **R** `BiocParallel::MulticoreParam` or `BiocParallel::SnowParam` object to combine with @parallelization = `bpmapply`.

To get detailed information of the input and output of the function, please check [API](../set_up/_autosummary/pyscDesign3.tools.get_bpparam.get_bpparam.rst).

To get more information on `BiocParallel` **R** package to help you set the parameters, please check the [document](https://www.bioconductor.org/packages/devel/bioc/manuals/BiocParallel/man/BiocParallel.pdf).